<a href="https://colab.research.google.com/github/dev-sampsonorson/chibuzor/blob/main/assignment-1/Assignment_1_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step 1: Load data into Google Colab

In [24]:
import pandas as pd
from tabulate import tabulate

url_csv = 'https://raw.githubusercontent.com/dev-sampsonorson/chibuzor/main/assignment-1/Part1.csv'

df = pd.read_csv(url_csv)

### Step 2: Explore the dataset
**Display the dataframe**

In [ ]:
# top 5, bottom 5
df

,ID,Insurance Type,Income of Policy Holder,Marital Status,Num Claimants,Injury Type,Overnight Hospital Stay,Claim Amount,Total Claimed,Num Claims,Num Soft Tissue,% Soft Tissue,Claim Amount Received,Fraud Flag
0,1,CI,0,NaN,2,Soft Tissue,No,1625,3250,2,2.0,1.0,0,1
1,2,CI,0,NaN,2,Back,Yes,15028,60112,1,0.0,0.0,15028,0
2,3,CI,54613,Married,1,Broken Limb,No,-99999,0,0,0.0,0.0,572,0
3,4,CI,0,NaN,3,Serious,Yes,270200,0,0,0.0,0.0,270200,0
4,5,CI,0,NaN,4,Soft Tissue,No,8869,0,0,0.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,CI,0,NaN,1,Soft Tissue,No,2118,0,0,0.0,0.0,0,1
496,497,CI,29280,Married,4,Broken Limb,Yes,3199,0,0,NaN,0.0,0,1
497,498,CI,0,NaN,1,Broken Limb,Yes,32469,0,0,0.0,0.0,16763,0
498,499,CI,46683,Married,1,Broken Limb,No,179448,0,0,0.0,0.0,179448,0


In [ ]:
# display all rows in an easy to read format
print(tabulate(df, headers = 'keys', tablefmt = 'grid'))

**Columns & data types in DataFrame**


In [ ]:
df.dtypes

ID                           int64
Insurance Type              object
Income of Policy Holder      int64
Marital Status              object
Num Claimants                int64
Injury Type                 object
Overnight Hospital Stay     object
Claim Amount                 int64
Total Claimed                int64
Num Claims                   int64
Num Soft Tissue            float64
% Soft Tissue              float64
Claim Amount Received        int64
Fraud Flag                   int64
dtype: object

**Display data (Series) of a single column**

In [ ]:
insurance_type_series = df['Insurance Type ']
# insurance_type_series.array
print(tabulate(pd.DataFrame(insurance_type_series.to_list(), columns = ['Insurance Type']), headers='keys', tablefmt='grid'))

**Add new column "Revenue"**

In [ ]:
df['Revenue'] = df["Num Claimants"] + df["Claim Amount Received"]
print(tabulate(pd.DataFrame(df, columns = ["Num Claimants", "Claim Amount Received", "Revenue"]), headers='keys', tablefmt='grid'))

**Select and display a specific row**

In [ ]:
df.iloc[0:1, 0:5]

,ID,Insurance Type,Income of Policy Holder,Marital Status,Num Claimants
0,1,CI,0,NaN,2


### Step 3: Identify missing attribute field(s). Which have missing values and how many? Propose a way to resolve these missing values for those attributes.

**Missing values and count**

In [33]:
df.isna().sum()

ID                           0
Insurance Type               0
Income of Policy Holder      0
Marital Status             330
Num Claimants                0
Injury Type                  0
Overnight Hospital Stay      0
Claim Amount                 0
Total Claimed                0
Num Claims                   0
Num Soft Tissue             10
% Soft Tissue                0
Claim Amount Received        0
Fraud Flag                   0
dtype: int64

**Propose a way to resolve these missing values**

*Marital Status*

To resolve this, i would suggest creating a new valid value called "Not Specifed", which would replace all the NaN values.

In [49]:
# Marital Status
# With values
len(df.loc[df['Marital Status'].isin(['Married', 'Single', 'Divorced']), "Marital Status"])

# With missing values
len(df.loc[~df['Marital Status'].isin(['Married', 'Single', 'Divorced']), "Marital Status"])



330

*Num Soft Tissue*

For this attribute i would suggest replacing the NaN values with 0.

In [54]:
df_nst = df['Num Soft Tissue'].fillna(0, inplace=False)
df_nst

0      2.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
495    0.0
496    0.0
497    0.0
498    0.0
499    0.0
Name: Num Soft Tissue, Length: 500, dtype: float64

In [ ]:
df[df['Insurance Type '].isna() | df['Insurance Type '].isnull(), "Insurance Type "]
df[df['Insurance Type '].isna() | df['Insurance Type '].isnull(), "Insurance Type "]
df[df['Insurance Type '].isna() | df['Insurance Type '].isnull(), "Insurance Type "]
df[df['Insurance Type '].isna() | df['Insurance Type '].isnull(), "Insurance Type "]
df[df['Insurance Type '].isna() | df['Insurance Type '].isnull(), "Insurance Type "]
df[df['Insurance Type '].isna() | df['Insurance Type '].isnull(), "Insurance Type "]
df[df['Insurance Type '].isna() | df['Insurance Type '].isnull(), "Insurance Type "]
df[df['Insurance Type '].isna() | df['Insurance Type '].isnull(), "Insurance Type "]
df[df['Insurance Type '].isna() | df['Insurance Type '].isnull(), "Insurance Type "]
df[df['Insurance Type '].isna() | df['Insurance Type '].isnull(), "Insurance Type "]
df[df['Insurance Type '].isna() | df['Insurance Type '].isnull(), "Insurance Type "]
df[df['Insurance Type '].isna() | df['Insurance Type '].isnull(), "Insurance Type "]
df[df['Insurance Type '].isna() | df['Insurance Type '].isnull(), "Insurance Type "]

In [ ]:
# df['Insurance Type '].to_list()

# insurance Type is odd because it has the same value all through
# meaning, the data for a paricular insurance type 
df.groupby(['Insurance Type ']).count()

# kkk = df.drop(['Insurance Type '], axis=1, inplace=False)
# kkk

,ID,Income of Policy Holder,Marital Status,Num Claimants,Injury Type,Overnight Hospital Stay,Claim Amount,Total Claimed,Num Claims,Num Soft Tissue,% Soft Tissue,Claim Amount Received,Fraud Flag
Insurance Type,,,,,,,,,,,,,
CI,500,500,170,500,500,500,500,500,500,490,500,500,500


In [ ]:
# columns with missing values
# df.value_counts(df["Marital Status"])

nan_count = len(df.loc[~df["Marital Status"].isin(["Married", "Single", "Divorced"]), "Marital Status"])
valid_count = len(df.loc[df["Marital Status"].isin(["Married", "Single", "Divorced"]), "Marital Status"])
print(nan_count, valid_count)
# x = df.loc[~df["Marital Status"].isin(["Married", "Single", "Divorced"]), ["ID", "Marital Status"]]
# print(tabulate(pd.DataFrame(x.to_dict(), columns = ['ID', 'Marital Status']), headers = 'keys', tablefmt = 'grid'))
# len(df.loc[df["Marital Status"].isna(), "Marital Status"])

330 170
